In [2]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [5]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk import download
from pyspark.ml.classification import LogisticRegression
import pyarrow as pa # for creating spark dataframe

In [6]:
download('stopwords')

[nltk_data] Downloading package stopwords to /home/muddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<a href="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf" target="_blank">Pandas cheat sheet</a><BR>

<A HREF="https://spark.apache.org/docs/latest/ml-guide.html" target="_blank">Pyspark ML guide</A><BR>

In [7]:
df = pd.read_csv("messages.csv")
messages = df.loc[:, 'message']
channel = df.loc[:, 'channel']
df.fillna('', inplace=True)

In [8]:
# Only for 2 channel's of data
# codes channel in 1st row as 0, other channel is 1
target = pd.DataFrame(np.where(df.channel==df.channel[0], 0, 1))
print(df.channel[0])

#summit1g


In [9]:
target.head()

,0
0,0
1,1
2,1
3,1
4,1


<a href="https://www.programiz.com/python-programming/list-comprehension" target="_blank">List comprehension</a><br>

In [81]:
#blobs = [TextBlob(str(message)) for message in messages ]

In [82]:
# POS - part of speach tagging - probably not needed here
#[blob.tags for blob in blobs][0:5]

In [83]:
print(messages[745:755])

745          Pog
746       xqcSus
747         PogU
748         PogU
749             
750    CLOSE DOR
751     PagChomp
752         PogU
753          Pog
754          lol
Name: message, dtype: object


In [10]:
# Stop word removal - and practice with list comprehension (see link above)
clean_txt = [[word.lower() for word in blob.split(',')[0].split() if word not in stopwords.words('english')] for blob in messages ]
clean_txt = [' '.join(txt) for txt in clean_txt]
blob_clean = [TextBlob(txt) for txt in clean_txt]
clean_text=pd.DataFrame(clean_txt)

In [11]:
blob_clean[0:5]

[TextBlob("monkaw cam talk"),
 TextBlob("ez"),
 TextBlob("ez"),
 TextBlob("ez"),
 TextBlob("pagman 󠀀")]

In [12]:
clean_txt[0:5]

['monkaw cam talk', 'ez', 'ez', 'ez', 'pagman \U000e0000']

In [ ]:
filtered_words = [word for word in word_tokenize(sent) if word not in stopwords.words('english')]

#### Ok, time to try a spark ML model. I'm working off the example <a href='https://spark.apache.org/docs/latest/ml-pipeline.html#code-examples'>here</a>

In [107]:
# move pandas df to spark df and tell it what are the features and what is the label
clean_text['target'] = target
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', "true")
training = spark.createDataFrame(clean_text, ["features", "label"])

In [108]:
training.head()

Row(features='monkaw cam talk', label=0)

In [ ]:
# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [109]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

IllegalArgumentException: requirement failed: Column features must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.StringType$:string.